In [1]:
import copy

# 外皮情報LV3からLV4へのコンバート

## I. 関数

### 1. 間仕切り床

#### 1) 床の仕様

| 番号 | 材料 | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> | 厚さ, m |
|---|---|---|---|---|
| 1 | 合板 | 0.16 | 720.0 | 0.024 |

In [2]:
def get_inner_floor_spec():
    
    return {
        'layers' : [
            {
                'name'                            : 'plywood',
                'thermal_resistance_input_method' : 'conductivity',
                'thermal_conductivity'            : 0.16,
                'volumetric_specific_heat'        : 720.0,
                'thickness'                       : 0.024,
            }
        ]
    }

#### 2) 方位が下面である外皮の部位の面積の合計

$$A_{evlp,down,MR} = \sum_i{A_{evlp,down,MR,i}}$$ 

$$A_{evlp,down,OR} = \sum_i{A_{evlp,down,OR,i}}$$ 

$$A_{evlp,down,NR} = \sum_i{A_{evlp,down,NR,i}}$$ 

$A_{evlp,down,MR}$：主たる居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,OR}$：その他の居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,NR}$：非居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,MR,i}$：主たる居室に接する方位が下面である外皮の部位iの面積, m<sup>2</sup>  
$A_{evlp,down,OR,i}$：その他の居室に接する方位が下面である外皮の部位iの面積, m<sup>2</sup>  
$A_{evlp,down,NR,i}$：非居室に接する方位が下面である外皮の部位iの面積, m<sup>2</sup>  

In [3]:
def get_downward_envelope_total_area(envelope):

    parts = []

    if 'general_parts' in envelope: parts.extend(envelope['general_parts']) 
    
    if 'windows' in envelope: parts.extend(envelope['windows'])
        
    if 'doors' in envelope: parts.extend(envelope['doors'])
    
    def f(space_type):
        return sum(part['area'] for part in parts
                   if part['space_type'] == space_type and (part['direction'] == 'bottom' or part['direction'] == 'downward'))

    return f('main_occupant_room'), f('other_occupant_room'), f('non_occupant_room')

#### 3) 土間床等の中心部の面積の合計

$$A_{ef,MR} = \sum_i {A_{ef,MR,i}}$$ 

$$A_{ef,OR}=\sum_i{A_{ef,OR,i}}$$

$$A_{ef,NR}=\sum_i{A_{ef,NR,i}}$$ 

$$A_{ef,uf}=\sum_i{A_{ef,uf,i}}$$ 

$A_{ef,MR}$：主たる居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,OR}$：その他の居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,NR}$：非居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,uf}$：床下空間に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,MR,i}$：主たる居室に接する土間床等の中心部$i$の面積, m<sup>2</sup>  
$A_{ef,OR,i}$：その他の居室に接する土間床等の中心部$i$の面積, m<sup>2</sup>  
$A_{ef,NR,i}$：非居室に接する土間床等の中心部$i$の面積, m<sup>2</sup>  
$A_{ef,uf}$：床下空間に接する土間床等の中心部$i$の面積, m<sup>2</sup>  

In [4]:
def get_earthfloor_total_area(envelope):

    parts = []
    
    if 'earthfloor_centers' in envelope: parts.extend(envelope['earthfloor_centers'])
    
    def f(space_type):
        return sum(part['area'] for part in parts
                   if part['space_type'] == space_type)

    return f('main_occupant_room'), f('other_occupant_room'), f('non_occupant_room'), f('underfloor')

#### 4) 間仕切り床の面積の合計

$$
A_{if,MR} = \max \left( 0, A_{MR} - A_{evlp,down,MR} - A_{ef,MR} \right)
$$

$$
A_{if,OR} = \max \left( 0, A_{OR} - A_{evlp,down,OR} - A_{ef,OR} \right)
$$

$$
A_{if,NR} = \max \left( 0, A_A - A_{MR} - A_{OR} - A_{evlp,down,NR} - A_{ef,NR} \right)
$$

$A_{if,MR}$：床上側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR}$：床上側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR}$：床上側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{A}$: 床面積の合計, m<sup>2</sup>   
$A_{evlp,down,MR}$：主たる居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,OR}$：その他の居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,down,NR}$：非居室に接する方位が下面である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{ef,MR}$：主たる居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,OR}$：その他の居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{ef,NR}$：非居室に接する土間床等の中心部の面積の合計, m<sup>2</sup>  

In [5]:
def get_inner_floor_total_area(a_a, a_mr, a_or,
                               a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr,
                               a_ef_mr, a_ef_or, a_ef_nr):

    a_if_mr = max(0.0, a_mr - a_evlp_down_mr - a_ef_mr)
    a_if_or = max(0.0, a_or - a_evlp_down_or - a_ef_or)
    a_if_nr = max(0.0, a_a - a_mr - a_or - a_evlp_down_nr - a_ef_nr)
    
    return a_if_mr, a_if_or, a_if_nr

#### 5) 間仕切り床の面積の合計（床下側が床下空間）

$A_{if,MR} + A_{if,OR} + A_{if,NR} > 0$の場合

$$A_{if,MR-uf} = A_{ef,uf} \frac{A_{if,MR}}{A_{if,MR} + A_{if,OR} + A_{if,NR}}$$

$$A_{if,OR-uf} = A_{ef,uf} \frac{A_{if,OR}}{A_{if,MR} + A_{if,OR} + A_{if,NR}}$$

$$A_{if,NR-uf} = A_{ef,uf} \frac{A_{if,NR}}{A_{if,MR} + A_{if,OR} + A_{if,NR}}$$

$A_{if,MR} + A_{if,OR} + A_{if,NR} = 0$の場合

$$A_{if,MR-uf}=0$$

$$A_{if,OR-uf}=0$$

$$A_{if,NR-uf}=0$$

$A_{if,MR-uf}$：床上側が主たる居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-uf}$：床上側がその他の居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-uf}$：床上側が非居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{ef,uf}$：床下空間に接する土間床等の中心部の面積の合計, m<sup>2</sup>  
$A_{if,MR}$：床上側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR}$：床上側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR}$：床上側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  

In [6]:
def get_inner_floor_over_underfloor_total_area(a_if_mr, a_if_or, a_if_nr, a_ef_uf):
    
    if (a_if_mr + a_if_or + a_if_nr) > 0.0:
        a_if_mr_uf = a_ef_uf * a_if_mr / (a_if_mr + a_if_or + a_if_nr)
        a_if_or_uf = a_ef_uf * a_if_or / (a_if_mr + a_if_or + a_if_nr)
        a_if_nr_uf = a_ef_uf * a_if_nr / (a_if_mr + a_if_or + a_if_nr)
    else :
        a_if_mr_uf = 0.0
        a_if_or_uf = 0.0
        a_if_nr_uf = 0.0
    
    return a_if_mr_uf, a_if_or_uf, a_if_nr_uf

#### 6) 間仕切り床の面積の合計（床下側が主たる居室・その他の居室・非居室）

$$ A_{if,MR-OR,NR} = \max{ \left( 0, A_{if,MR}-A_{if,MR-uf} \right) } $$

$$ A_{if,OR-MR,NR} = \max{ \left( 0, A_{if,OR}-A_{if,OR-uf} \right) } $$

$$ A_{if,NR-MR,OR} = \max{ \left( 0, A_{if,NR}-A_{if,NR-uf} \right) } $$

$A_{OR}=0$かつ$A_{NR}=0$の場合

$$A_{if,MR-OR}=0$$

$$A_{if,MR-NR}=0$$

$$A_{if,OR-MR}=0$$

$$A_{if,OR-NR}=0$$

$$A_{if,NR-MR}=0$$

$$A_{if,NR-OR}=0$$

$A_{OR}=0$かつ$A_{NR}\ne0$の場合

$$A_{if,MR-OR}=0$$

$$A_{if,MR-NR}=A_{if,MR-OR,NR}$$

$$A_{if,OR-MR}=0$$

$$A_{if,OR-NR}=0$$

$$A_{if,NR-MR}=A_{if,NR-MR,OR}$$

$$A_{if,NR-OR}=0$$

$A_{OR}\ne0$かつ$A_{NR}=0$の場合

$$A_{if,MR-OR}=A_{if,MR-OR,NR}$$

$$A_{if,MR-NR}=0$$

$$A_{if,NR-MR}=0$$

$$A_{if,NR-OR}=0$$

$$A_{if,OR-MR}=A_{if,OR-MR,NR}$$

$$A_{if,OR-NR}=0$$

$A_{OR}\ne0$かつ$A_{NR}\ne0$の場合

$$A_{if,MR-OR} = \frac{ A_{if,MR-OR,NR} }{ 2 }$$

$$A_{if,MR-NR} = \frac{ A_{if,MR-OR,NR} }{ 2 }$$

$$A_{if,OR-MR} = \frac{ A_{if,OR-MR,NR} }{ 2 }$$

$$A_{if,OR-NR} = \frac{ A_{if,OR-MR,NR} }{ 2 }$$

$$A_{if,NR-MR} = \frac{ A_{if,NR-MR,OR} }{ 2 }$$

$$A_{if,NR-OR} = \frac{ A_{if,NR-MR,OR} }{ 2 }$$

$A_{if,MR-OR,NR}$:床上側が主たる居室に接し床下側がその他の居室又は非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-MR,NR}$:床上側がその他の居室に接し床下側が主たる居室又は非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-MR,OR}$:床上側が非居室に接し床下側が主たる居室又はその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR}$:床上側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR}$:床上側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR}$:床上側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR-uf}$:床上側が主たる居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-uf}$:床上側がその他の居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-uf}$:床上側が非居室に接し床下側が床下空間に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR-OR}$:床上側が主たる居室に接し床下側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,MR-NR}$:床上側が主たる居室に接し床下側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-MR}$:床上側がその他の居室に接し床下側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,OR-NR}$:床上側がその他の居室に接し床下側が非居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-MR}$:床上側が非居室に接し床下側が主たる居室に接する間仕切り床の面積の合計, m<sup>2</sup>  
$A_{if,NR-OR}$:床上側が非居室に接し床下側がその他の居室に接する間仕切り床の面積の合計, m<sup>2</sup>  

In [7]:
def get_inner_floor_between_rooms(a_mr, a_or, a_a, a_if_mr, a_if_or, a_if_nr, a_if_mr_uf, a_if_or_uf, a_if_nr_uf):
    
    a_nr = a_a - a_mr - a_or
    
    if a_mr <= 0.0:
        raise ValueError('主たる居室の面積に0又は負の値が指定されました。主たる居室は必ず0より大きい値を指定する必要があります。')
    
    a_if_mr_or_nr = max(0.0, a_if_mr - a_if_mr_uf)
    a_if_or_mr_nr = max(0.0, a_if_or - a_if_or_uf)
    a_if_nr_mr_or = max(0.0, a_if_nr - a_if_nr_uf)

    if a_or <= 0.0 and a_nr <= 0.0:
        a_if_mr_or = 0.0
        a_if_mr_nr = 0.0
        a_if_or_mr = 0.0
        a_if_or_nr = 0.0
        a_if_nr_mr = 0.0
        a_if_nr_or = 0.0
    elif a_or <= 0.0:
        a_if_mr_or = 0.0
        a_if_mr_nr = a_if_mr_or_nr
        a_if_or_mr = 0.0
        a_if_or_nr = 0.0
        a_if_nr_mr = a_if_nr_mr_or
        a_if_nr_or = 0.0
    elif a_nr <= 0.0:
        a_if_mr_or = a_if_mr_or_nr
        a_if_mr_nr = 0.0
        a_if_or_mr = a_if_or_mr_nr
        a_if_or_nr = 0.0
        a_if_nr_mr = 0.0
        a_if_nr_or = 0.0
    else:
        a_if_mr_or = a_if_mr_or_nr / 2
        a_if_mr_nr = a_if_mr_or_nr / 2
        a_if_or_mr = a_if_or_mr_nr / 2
        a_if_or_nr = a_if_or_mr_nr / 2
        a_if_nr_mr = a_if_nr_mr_or / 2
        a_if_nr_or = a_if_nr_mr_or / 2
    
    return a_if_mr_or, a_if_mr_nr, a_if_or_mr, a_if_or_nr, a_if_nr_mr, a_if_nr_or

### 2. 間仕切り壁

#### 1) 壁の仕様

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup>2 k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| せっこうボード | 0.0125 | 0.22 | - | 830.0 |
| 非密閉空気層 | - | - | 0.090 | 0.0 |
| せっこうボード | 0.0125 | 0.22 | - | 830.0 |

In [8]:
def get_inner_wall_spec():
    return {
        'layers' : [
            {
                'name'                            : 'gpb',
                'thermal_resistance_input_method' : 'conductivity',
                'thermal_conductivity'            : 0.22,
                'thickness'                       : 0.0125,
                'volumetric_specific_heat'        : 830.0,
            },
            {
                'name'                            : 'air_layer',
                'thermal_resistance_input_method' : 'resistance',
                'thermal_resistance'              : 0.09,
                'volumetric_specific_heat'        : 0.0,
            },
            {
                'name'                            : 'gpb',
                'thermal_resistance_input_method' : 'conductivity',
                'thermal_conductivity'            : 0.22,
                'thickness'                       : 0.0125,
                'volumetric_specific_heat'        : 830.0,
            },
        ]
    }

#### 2) 方位が水平である外皮の部位の面積の合計

$$ A_{evlp,hzt,MR} = \sum_i{ A_{evlp,hzt,MR,i} } $$

$$ A_{evlp,hzt,OR} = \sum_i{ A_{evlp,hzt,OR,i} } $$

$$ A_{evlp,hzt,NR} = \sum_i{ A_{evlp,hzt,NR,i} } $$

$ A_{evlp,hzt,MR} $:主たる居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  
$ A_{evlp,hzt,OR} $:その他の居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  
$ A_{evlp,hzt,NR} $:非居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  
$ A_{evlp,hzt,MR,i} $:主たる居室に接する方位が水平である外皮の部位$i$の面積, m<sup>2</sup>  
$ A_{evlp,hzt,OR,i} $:その他の居室に接する方位が水平である外皮の部位$i$の面積, m<sup>2</sup>  
$ A_{evlp,hzt,NR,i} $:非居室に接する方位が水平である外皮の部位$i$の面積, m<sup>2</sup>  

In [9]:
def get_horizontal_envelope_total_area(envelope):

    parts = []

    if 'general_parts' in envelope: parts.extend(envelope['general_parts'])
        
    if 'windows' in envelope: parts.extend(envelope['windows'])
        
    if 'doors' in envelope: parts.extend(envelope['doors'])

    def is_horizontal(part):
        d = part['direction']
        return d == 'n' or d == 'ne' or d == 'e' or d == 'se' or \
               d == 's' or d == 'sw' or d == 'w' or d == 'nw' or \
               d == 'horizontal'
        
    def f(space_type):
        return sum(part['area'] for part in parts \
                   if part['space_type'] == space_type and is_horizontal(part))
    
    return f('main_occupant_room'), f('other_occupant_room'), f('non_occupant_room')

#### 3) 間仕切り壁の面積の合計

$$ A_{iw,MR} = \max{ \left( 0, 4 r_{iw,MR} h_{iw} \sqrt{ A_{MR} } - A_{evlp,hzt,MR} \right) } $$

$$ A_{iw,OR} = \max⁡{ \left( 0, 4 r_{iw,OR} h_{iw} \sqrt{ A_{OR} } - A_{evlp,hzt,OR} \right) } $$

$$ A_{iw,NR} = \max⁡{ \left( 0, 4 r_{iw,NR} h_{iw} \sqrt{ A_A - A_{MR} - A_{OR} } - A_{evlp,hzt,NR} \right) } $$

$A_{iw,MR}$:主たる居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,OR}$:その他の居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,NR}$:非居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$r_{iw,MR}$:主たる居室の周長を求めるための形状係数($=1.2$)  
$r_{iw,OR}$:その他の居室の周長を求めるための形状係数($=1.4$)  
$r_{iw,NR}$:非居室の周長を求めるための形状係数($=1.4$)  
$h_{iw}$:階高($=2.8$), m  
$A_{MR}$:主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$:その他の居室の床面積, m<sup>2</sup>  
$A_A$:床面積の合計, m<sup>2</sup>  
$A_{evlp,hzt,MR}$:主たる居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,hzt,OR}$:その他の居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  
$A_{evlp,hzt,NR}$:非居室に接する方位が水平である外皮の部位の面積の合計, m<sup>2</sup>  

In [10]:
def get_inner_wall_total_area(a_mr, a_or, a_a, a_evlp_hzt_mr, a_evlp_hzt_or, a_evlp_hzt_nr):
    
    r_iw_mr, r_iw_or, r_iw_nr = 1.2, 1.4, 1.4

    h_iw = 2.8
    
    a_iw_mr = max( 0.0, 4 * r_iw_mr * h_iw *            a_mr**0.5 - a_evlp_hzt_mr )
    a_iw_or = max( 0.0, 4 * r_iw_or * h_iw *            a_or**0.5 - a_evlp_hzt_or )
    a_iw_nr = max( 0.0, 4 * r_iw_nr * h_iw * (a_a-a_mr-a_or)**0.5 - a_evlp_hzt_nr )
    
    return a_iw_mr, a_iw_or, a_iw_nr

#### 4) 各用途間の間仕切り壁の面積の合計

$ A_{OR} = 0 $ かつ $ A_A - A_{MR} - A_{OR} = 0 $の場合

$$ A_{iw,MR-OR} = 0 $$

$$ A_{iw,MR-NR} = 0 $$

$$ A_{iw,OR-NR} = 0 $$

$ A_{OR} = 0 $ かつ $ A_A - A_{MR} - A_{OR} \ne 0 $ の場合

$$ A_{iw,MR-OR} = 0 $$

$$ A_{iw,MR-NR} = \frac{ \left( A_{iw,MR} + A_{iw,NR} \right) }{ 2 } $$

$$ A_{iw,OR-NR} = 0 $$

$ A_{OR} \ne 0 $ かつ $ A_A - A_{MR} - A_{OR} = 0 $ の場合

$$ A_{iw,MR-OR} = 0 $$

$$ A_{iw,MR-NR} = 0 $$

$$ A_{iw,OR-NR} = \frac{ \left( A_{iw,OR} + A_{iw,NR} \right) }{ 2 } $$

$ A_{OR} \ne 0 $ かつ $ A_A - A_{MR} - A_{OR} \ne 0 $ の場合

$$ A_{iw,MR-OR} = \max{ \left( 0.0, \frac{ \left( A_{iw,MR} + A_{iw,OR} -A_{iw,NR} \right) }{ 2 } \right) } $$

$$ A_{iw,MR-NR} = \max{ \left( 0.0, \frac{ \left( A_{iw,MR} + A_{iw,NR} -A_{iw,OR} \right) }{ 2 } \right) } $$

$$ A_{iw,OR-NR} = \max{ \left( 0.0, \frac{ \left( A_{iw,OR} + A_{iw,NR} -A_{iw,MR} \right) }{ 2 } \right) } $$

$A_{MR}$:主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$:その他の居室の床面積, m<sup>2</sup>  
$A_A$:床面積の合計, m<sup>2</sup>  
$A_{iw,MR-OR}$:主たる居室とその他の居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,MR-NR}$:主たる居室と非居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,OR-NR}$:その他の居室と非居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,MR}$:主たる居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,OR}$:その他の居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  
$A_{iw,NR}$:非居室に接する間仕切り壁の面積の合計, m<sup>2</sup>  

In [11]:
def get_inner_wall_total_area_between_rooms(a_mr, a_or, a_a, a_iw_mr, a_iw_or, a_iw_nr):

    a_nr = a_a - a_mr - a_or
    
    if a_mr <= 0.0:
        raise ValueError('主たる居室の面積に0又は負の値が指定されました。主たる居室は必ず0より大きい値を指定する必要があります。')
    
    if a_or <= 0.0 and a_nr <= 0.0:
        a_iw_mr_or = 0.0
        a_iw_mr_nr = 0.0
        a_iw_or_nr = 0.0
    elif a_or <= 0.0:
        a_iw_mr_or = 0.0
        a_iw_mr_nr = (a_iw_mr + a_iw_nr)/2
        a_iw_or_nr = 0.0
    elif a_nr <= 0.0:
        a_iw_mr_or = (a_iw_mr + a_iw_or)/2
        a_iw_mr_nr = 0.0
        a_iw_or_nr = 0.0
    else:
        a_iw_mr_or = max(0.0, (a_iw_mr + a_iw_or - a_iw_nr) / 2)
        a_iw_mr_nr = max(0.0, (a_iw_mr + a_iw_nr - a_iw_or) / 2)
        a_iw_or_nr = max(0.0, (a_iw_or + a_iw_nr - a_iw_mr) / 2)
    
    return a_iw_mr_or, a_iw_mr_nr, a_iw_or_nr

#### 3) 内壁の構築

In [12]:
def get_innerwall_floor(d):

    a_mr = d['common']['main_occupant_room_floor_area']
    a_or = d['common']['other_occupant_room_floor_area']
    a_a  = d['common']['total_floor_area']

    a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr = get_downward_envelope_total_area(d)
    
    a_ef_mr, a_ef_or, a_ef_nr, a_ef_uf = get_earthfloor_total_area(d)

    a_if_mr, a_if_or, a_if_nr = get_inner_floor_total_area(a_a, a_mr, a_or,
                                                           a_evlp_down_mr, a_evlp_down_or, a_evlp_down_nr,
                                                           a_ef_mr, a_ef_or, a_ef_nr)

    a_if_mr_uf, a_if_or_uf, a_if_nr_uf = get_inner_floor_over_underfloor_total_area(a_if_mr, a_if_or, a_if_nr, a_ef_uf)    

    a_if_mr_or, a_if_mr_nr, a_if_or_mr, a_if_or_nr, a_if_nr_mr, a_if_nr_or \
    = get_inner_floor_between_rooms(a_mr, a_or, a_a, a_if_mr, a_if_or, a_if_nr, a_if_mr_uf, a_if_or_uf, a_if_nr_uf)

    ifs = []
    
    def append_inner_floor(ifs, name, area, upper_space_type, lower_space_type):
        
        if area > 0.0:
            ifs.append(
                {
                    'name'             : name,
                    'area'             : area,
                    'upper_space_type' : upper_space_type,
                    'lower_space_type' : lower_space_type,
                    'spec'             : get_inner_floor_spec()
                }
            )

    append_inner_floor(ifs, 'inner_floor_main_to_underfloor',  a_if_mr_uf, 'main_occupant_room',  'underfloor'          )
    append_inner_floor(ifs, 'inner_floor_other_to_underfloor', a_if_or_uf, 'other_occupant_room', 'underfloor'          )
    append_inner_floor(ifs, 'inner_floor_non_to_underfloor',   a_if_nr_uf, 'non_occupant_room',   'underfloor'          )
    append_inner_floor(ifs, 'inner_floor_main_to_other',       a_if_mr_or, 'main_occupant_room',  'other_occupant_room' )
    append_inner_floor(ifs, 'inner_floor_main_to_non',         a_if_mr_nr, 'main_occupant_room',  'non_occupant_room'   )
    append_inner_floor(ifs, 'inner_floor_other_to_main',       a_if_or_mr, 'other_occupant_room', 'main_occupant_room'  )
    append_inner_floor(ifs, 'inner_floor_other_to_non',        a_if_or_nr, 'other_occupant_room', 'non_occupant_room'   )
    append_inner_floor(ifs, 'inner_floor_non_to_main',         a_if_nr_mr, 'non_occupant_room',   'main_occupant_room'  )
    append_inner_floor(ifs, 'inner_floor_non_to_other',        a_if_nr_or, 'non_occupant_room',   'other_occupant_room' )
                    
    return ifs

In [13]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        {
            'name' : 'floor',
            'evlp_type' : 'floor',
            'next_space' : 'open_underfloor',
            'direction' : 'bottom',
            'area' : 60.0,
            'space_type': 'main',
            'spec' : 'something'
        },
    ],
    'earthfloor_centers': [
        { 'name': 'other',    'area': 5.0, 'space_type': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
    ]
}

get_innerwall_floor(d)

[{'name': 'inner_floor_main_to_underfloor',
  'area': 2.5,
  'upper_space_type': 'main_occupant_room',
  'lower_space_type': 'underfloor',
  'spec': {'layers': [{'name': 'plywood',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.16,
     'volumetric_specific_heat': 720.0,
     'thickness': 0.024}]}},
 {'name': 'inner_floor_other_to_underfloor',
  'area': 5.0,
  'upper_space_type': 'other_occupant_room',
  'lower_space_type': 'underfloor',
  'spec': {'layers': [{'name': 'plywood',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.16,
     'volumetric_specific_heat': 720.0,
     'thickness': 0.024}]}},
 {'name': 'inner_floor_non_to_underfloor',
  'area': 2.5,
  'upper_space_type': 'non_occupant_room',
  'lower_space_type': 'underfloor',
  'spec': {'layers': [{'name': 'plywood',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.16,
     'volumetric_specific_heat': 720.0,
     '

##### Function

In [14]:
def get_innerwall_wall(d):

    a_mr = d['common']['main_occupant_room_floor_area']
    a_or = d['common']['other_occupant_room_floor_area']
    a_a  = d['common']['total_floor_area']
 
    a_ow_mr, a_ow_or, a_ow_nr = get_horizontal_envelope_total_area(d)

    a_iw_mr, a_iw_or, a_iw_nr = get_inner_wall_total_area(a_mr, a_or, a_a, a_ow_mr, a_ow_or, a_ow_nr)
        
    a_iw_mr_or, a_iw_mr_nr, a_iw_or_nr = get_inner_wall_total_area_between_rooms(a_mr, a_or, a_a, a_iw_mr, a_iw_or, a_iw_nr)
    
    iws = []

    def append_inner_wall(iws, name, area, space_type_1, space_type_2):
        
        if area > 0.0:
            iws.append(
                {
                    'name'         : name,
                    'area'         : area,
                    'space_type_1' : space_type_1,
                    'space_type_2' : space_type_2,
                    'spec'         : get_inner_wall_spec()
                }
            )

    append_inner_wall(iws, 'inner_wall_main_to_other', a_iw_mr_or, 'main_occupant_room', 'other_occupant_room')
    append_inner_wall(iws, 'inner_wall_main_to_non',   a_iw_mr_nr, 'main_occupant_room', 'non_occupant_room')
    append_inner_wall(iws, 'inner_wall_other_to_non',  a_iw_or_nr, 'other_occupant_room', 'non_occupant_room')

    return iws

##### Example

In [15]:
correspondence = {
    'main to other': ['main', 'other'],
    'main to nonliving': ['main', 'nonliving'],
    'other to nonliving': ['other', 'nonliving'] 
}  

d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space_type': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space_type': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space_type': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'earthfloor_centers': [
        { 'name': 'other', 'area': 5.0, 'space_type': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
]

get_innerwall_wall(d)

[{'name': 'inner_wall_main_to_other',
  'area': 54.59388622447443,
  'space_type_1': 'main_occupant_room',
  'space_type_2': 'other_occupant_room',
  'spec': {'layers': [{'name': 'gpb',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.22,
     'thickness': 0.0125,
     'volumetric_specific_heat': 830.0},
    {'name': 'air_layer',
     'thermal_resistance_input_method': 'resistance',
     'thermal_resistance': 0.09,
     'volumetric_specific_heat': 0.0},
    {'name': 'gpb',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.22,
     'thickness': 0.0125,
     'volumetric_specific_heat': 830.0}]}},
 {'name': 'inner_wall_main_to_non',
  'area': 19.020025504219888,
  'space_type_1': 'main_occupant_room',
  'space_type_2': 'non_occupant_room',
  'spec': {'layers': [{'name': 'gpb',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.22,
     'thickness': 0.0125,
     'volumetric_specif

### 8.3. InnerWalls Element - 統合

##### Function

In [16]:
def convert_innerwall(d):

    # 室と床下空間の間の床の推定
    iws_floor = get_innerwall_floor(d)
    
    # 内壁等の推定              
    iws_wall = get_innerwall_wall(d)
    
    return iws_floor + iws_wall

##### Example

In [17]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 60, 'space_type': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space_type': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 60, 'space_type': 'main',
          'FloorConstructionMethod' :'FrameInsulcolumn',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False }
    ],
    'windows': [
        { 'name': 'WindowTop', 'direction': 'top', 'area': 30, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji', 'IsSunshadeInput': False }
    ],
    'earthfloor_centers': [
        { 'name': 'other', 'area': 5.0, 'space_type': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
    ]
}

innerwalls = [
        { 'area': 60, 'name': 'GroundFloor_Main', 'type' : 'GroundFloor', 'spaces': ['main','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
        { 'area': 15, 'name': 'GroundFloor_Other', 'type' : 'GroundFloor', 'spaces': ['other','underfloor'], 'direction' : 'Horizontal', 'Layers' : get_inner_floor_spec()},
]

convert_innerwall( d )

[{'name': 'inner_floor_main_to_underfloor',
  'area': 2.5,
  'upper_space_type': 'main_occupant_room',
  'lower_space_type': 'underfloor',
  'spec': {'layers': [{'name': 'plywood',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.16,
     'volumetric_specific_heat': 720.0,
     'thickness': 0.024}]}},
 {'name': 'inner_floor_other_to_underfloor',
  'area': 5.0,
  'upper_space_type': 'other_occupant_room',
  'lower_space_type': 'underfloor',
  'spec': {'layers': [{'name': 'plywood',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.16,
     'volumetric_specific_heat': 720.0,
     'thickness': 0.024}]}},
 {'name': 'inner_floor_non_to_underfloor',
  'area': 2.5,
  'upper_space_type': 'non_occupant_room',
  'lower_space_type': 'underfloor',
  'spec': {'layers': [{'name': 'plywood',
     'thermal_resistance_input_method': 'conductivity',
     'thermal_conductivity': 0.16,
     'volumetric_specific_heat': 720.0,
     '

## 9. Lv4入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床、8)内壁等の各コンバート関数を統合し、Lv3の入力情報をLv4の入力情報にコンバートする。

##### Function

In [18]:
def convert(d):
    
    d_lv4_input = {}
    
    d_lv4_input['common'] = copy.deepcopy(d['common'])
    
    if ('general_parts' in d) == True:
        d_lv4_input['general_parts'] = copy.deepcopy(d['general_parts'])
        
    if ('windows' in d) == True:
        d_lv4_input['windows'] = copy.deepcopy(d['windows'])
        
    if ('doors' in d) == True:
        d_lv4_input['doors'] = copy.deepcopy(d['doors'])
        
    if ('heatbridges' in d) == True:
        d_lv4_input['heatbridges'] = copy.deepcopy(d['heatbridges'])
        
    if ('earthfloor_perimeters' in d) == True:
        d_lv4_input['earthfloor_perimeters'] = copy.deepcopy(d['earthfloor_perimeters'])
        
    if ('earthfloor_centers' in d) == True:
        d_lv4_input['earthfloor_centers'] = copy.deepcopy(d['earthfloor_centers'])
        
    d_lv4_input['InnerWalls'] = convert_innerwall(d)

    return d_lv4_input

##### Example

In [19]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'InputMethod' :'InputUA', 
          'direction': 'top', 'area': 67.8, 'space_type': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space_type': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'space_type': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 67.8,
          'ConstructionMethodAllLayers' :'FloorFrameBeamInsuljoist',
          'Parts': [{'Type': 'FloorInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'FloorHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False },
        { 'name': 'Wall', 'type': 'Wall', 'space_type': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'N',
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'WallFrameInsulcolumn',
          'Parts': [{'Type': 'WallInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'WallHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Roof', 'space_type': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'top',
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'RoofInsulRafter', 
          'Parts': [{'Type': 'RoofInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'RoofHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'BoundaryCeiling', 'type': 'BoundaryCeiling', 'space_type': 'main', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'ConstructionMethodAllLayers': 'CeilingInsulbeam', 
          'Parts': [{'Type': 'CeilingInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'Type': 'CeilingHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUR', 'InsulationMethod': 'CeilingInsulbeam',
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space_type': 'main', 
          'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                 {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod' :'InputLayers', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space_type': 'main',
          'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                 {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod' :'InputUR', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space_type': 'main', 'Radding': 0.10, 
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'other', 'direction': 'top', 'IsInContactWithOutsideAir': True, 
          'area': 67.8, 'space_type': 'main', 'IsSunshadeInput': False }
    ],
    'windows': [
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'EtaInputMethod': 'InputSpecification', 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'space_type': 'main', 'UW': 4.65, 'TypeWindow': 'Single', 
          'EtaInputMethod': 'InputValue', 'Eta': 0.738, 'IsSunshadeInput': False },
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'space_type': 'main', 'UW': 6.51, 'TypeWindow': 'Double', 
          'EtaInputMethod': 'InputSpecification', 'TypeFrameInside': 'WoodOrResin', 'TypeGlassInside': '3WgG', 'TypeShadeInside': 'Shoji',
          'TypeFrameOutside': 'Steel', 'TypeGlassOutside': '3WgG', 'TypeShadeOutside': 'ExtarnalBlind',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'space_type': 'main', 'UW': 4.65, 'TypeWindow': 'Double',
          'EtaInputMethod': 'InputSpecification', 'EtaInside': 0.738, 'TypeGlassInside': '3WgG', 'EtaOutside': 0.738, 
          'TypeGlassOutside': '3WgG', 'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'doors': [
        { 'name': 'DoorNW', 'direction': 'NW', 'area': 2.52, 'U': 6.51, 'space_type': 'main', 'IsSunshadeInput': False },
        { 'name': 'DoorNE', 'direction': 'NE', 'area': 2.16, 'U': 4.65, 'space_type': 'main', 
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'heatbridges': [
        { 'name': 'NE', 'structure': 'RC', 'length': 1.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'E', 'space_type': 'main' },
        { 'name': 'NW', 'structure': 'Steel', 'length': 2.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'W', 'space_type': 'main' }
    ],
    'earthfloor_perimeters': [
        { 'name': 'NW', 'direction': 'NW', 'length': 2.43, 'psi': 1.8, 'space_type': 'underfloor' },
        { 'name': 'NE', 'direction': 'NE', 'length': 2.43, 'psi': 1.8, 'space_type': 'underfloor' }
    ],
    'earthfloor_centers': [
        { 'name': 'other', 'area': 5.0, 'space_type': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor' }
    ]
}

convert(d)

{'common': {'region': 6,
  'main_occupant_room_floor_area': 30.0,
  'other_occupant_room_floor_area': 60.0,
  'total_floor_area': 120.0},
 'general_parts': [{'name': 'Ceiling',
   'type': 'Ceiling',
   'InputMethod': 'InputUA',
   'direction': 'top',
   'area': 67.8,
   'space_type': 'main',
   'UA': 0.24,
   'IsSunshadeInput': False},
  {'name': 'Ceiling',
   'type': 'Ceiling',
   'structure': 'wood',
   'InputMethod': 'InputAllDetails',
   'direction': 'top',
   'IsInContactWithOutsideAir': True,
   'area': 67.8,
   'space_type': 'main',
   'Parts': [{'AreaRatio': 0.8,
     'Layers': [{'name': 'wood', 'thick': 0.012, 'cond': 0.16, 'specH': 720},
      {'name': 'wood', 'thick': 0.012, 'cond': 0.16, 'specH': 720}]},
    {'AreaRatio': 0.2,
     'Layers': [{'name': 'wood', 'thick': 0.012, 'cond': 0.16, 'specH': 720},
      {'name': 'wood', 'thick': 0.012, 'cond': 0.16, 'specH': 720}]}],
   'IsSunshadeInput': False},
  {'name': 'Floor',
   'type': 'Floor',
   'space_type': 'main',
   'str

In [20]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 30.0,
        'total_floor_area': 120.0
    },
    'general_parts': [
        {'name': 'Ceiling_main', 'direction': 'Top', 'area': 16.95, 'space_type': 'main', 'type': 'Ceiling',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Ceiling_other', 'direction': 'Top', 'area': 16.95, 'space_type': 'other', 'type': 'Ceiling',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Ceiling_nonliving', 'direction': 'Top', 'area': 33.9, 'space_type': 'nonliving', 'type': 'Ceiling',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 7.7},
        {'name': 'Wall_SW_main', 'direction': 'SW', 'area': 10.1575, 'space_type': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SW_other', 'direction': 'SW', 'area': 10.1575, 'space_type': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SW_nonliving', 'direction': 'SW', 'area': 20.315, 'space_type': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_main', 'direction': 'NW', 'area': 7.4575, 'space_type': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_other', 'direction': 'NW', 'area': 7.4575, 'space_type': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NW_nonliving', 'direction': 'NW', 'area': 14.915, 'space_type': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_main', 'direction': 'NE', 'area': 15.9725, 'space_type': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_other', 'direction': 'NE', 'area': 15.9725, 'space_type': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_NE_nonliving', 'direction': 'NE', 'area': 31.945, 'space_type': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_main', 'direction': 'SE', 'area': 7.4275, 'space_type': 'main', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_other', 'direction': 'SE', 'area': 7.4275, 'space_type': 'other', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Wall_SE_nonliving', 'direction': 'SE', 'area': 14.855, 'space_type': 'nonliving', 'type': 'Wall',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 6.67},
        {'name': 'Floor_main', 'direction': 'Bottom', 'area': 15.0175, 'space_type': 'main', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_other', 'direction': 'Bottom', 'area': 15.0175, 'space_type': 'other', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_nonliving', 'direction': 'Bottom', 'area': 30.035, 'space_type': 'nonliving', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_main', 'direction': 'Bottom', 'area': 1.1025, 'space_type': 'main', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_other', 'direction': 'Bottom', 'area': 1.1025, 'space_type': 'other', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27},
        {'name': 'Floor_bath_nonliving', 'direction': 'Bottom', 'area': 2.205, 'space_type': 'nonliving', 'type': 'Floor',
         'structure': 'wood', 'IsSunshadeInput': False, 'UA': 5.27}
    ],
    'windows': [
        {'name': 'WindowSW_main', 'direction': 'SW', 'area': 7.5625, 'space_type': 'main', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSW_other', 'direction': 'SW', 'area': 7.5625, 'space_type': 'other', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSW_nonliving', 'direction': 'SW', 'area': 15.125, 'space_type': 'nonliving', 'UW': 3.49, 
         'EtaInputMethod': 'InputSeasonalValue', 'EtaCooling': 0.51, 'EtaHeating': 0.51,
         'IsSunshadeInput': True, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNW_main', 'direction': 'NW', 'area': 0.7925, 'space_type': 'main', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNW_other', 'direction': 'NW', 'area': 0.7925, 'space_type': 'other', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNW_nonliving', 'direction': 'NW', 'area': 1.585, 'space_type': 'nonliving', 'UW': 3.49, 
         'EtaInputMethod': 'InputSeasonalValue', 'EtaCooling': 0.51, 'EtaHeating': 0.51,
         'IsSunshadeInput': True, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNE_main', 'direction': 'NE', 'area': 1.21, 'space_type': 'main', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNE_other', 'direction': 'NE', 'area': 1.21, 'space_type': 'other', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowNE_nonliving', 'direction': 'NE', 'area': 2.42, 'space_type': 'nonliving', 'UW': 3.49, 
         'EtaInputMethod': 'InputSeasonalValue', 'EtaCooling': 0.51, 'EtaHeating': 0.51,
         'IsSunshadeInput': True, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSE_main', 'direction': 'SE', 'area': 1.4575, 'space_type': 'main', 'UW': 3.49, 'EtaInputMethod': 'InputSeasonalValue',
         'IsSunshadeInput': True, 'EtaCooling': 0.51, 'EtaHeating': 0.51, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSE_other', 'direction': 'SE', 'area': 1.4575, 'space_type': 'other', 'UW': 3.49, 
         'EtaInputMethod': 'InputSeasonalValue', 'EtaCooling': 0.51, 'EtaHeating': 0.51, 
         'IsSunshadeInput': True, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None},
        {'name': 'WindowSE_nonliving', 'direction': 'SE', 'area': 2.915, 'space_type': 'nonliving', 'UW': 3.49,
         'EtaInputMethod': 'InputSeasonalValue', 'EtaCooling': 0.51, 'EtaHeating': 0.51, 
         'IsSunshadeInput': True, 'TypeGlass': None, 'Y1': 0, 'Y2': 1.1, 'Z': None}
    ],
    'doors': [
        {'name': 'DoorNW_main', 'direction': 'NW', 'area': 0.63, 'space_type': 'main', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNW_other', 'direction': 'NW', 'area': 0.63, 'space_type': 'other', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNW_nonliving', 'direction': 'NW', 'area': 1.26, 'space_type': 'nonliving', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_main', 'direction': 'NE', 'area': 0.54, 'space_type': 'main', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_other', 'direction': 'NE', 'area': 0.54, 'space_type': 'other', 'U': 4.65, 'IsSunshadeInput': False},
        {'name': 'DoorNE_nonliving', 'direction': 'NE', 'area': 1.08, 'space_type': 'nonliving', 'U': 4.65, 'IsSunshadeInput': False}
    ],
    'earthfloor_perimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'Entrance_NW', 'psi': 1.8, 'space_type': 'underfloor'},
        {'direction': 'NE', 'length': 1.83, 'name': 'Entrance_NE', 'psi': 1.8, 'space_type': 'underfloor'},
        {'direction': 'OpenBackFloor', 'length': 4.25, 'name': 'Entrance_floor', 'psi': 1.8, 'space_type': 'underfloor'}
    ]
}

convert(d)

{'common': {'region': 6,
  'main_occupant_room_floor_area': 30.0,
  'other_occupant_room_floor_area': 30.0,
  'total_floor_area': 120.0},
 'general_parts': [{'name': 'Ceiling_main',
   'direction': 'Top',
   'area': 16.95,
   'space_type': 'main',
   'type': 'Ceiling',
   'structure': 'wood',
   'IsSunshadeInput': False,
   'UA': 7.7},
  {'name': 'Ceiling_other',
   'direction': 'Top',
   'area': 16.95,
   'space_type': 'other',
   'type': 'Ceiling',
   'structure': 'wood',
   'IsSunshadeInput': False,
   'UA': 7.7},
  {'name': 'Ceiling_nonliving',
   'direction': 'Top',
   'area': 33.9,
   'space_type': 'nonliving',
   'type': 'Ceiling',
   'structure': 'wood',
   'IsSunshadeInput': False,
   'UA': 7.7},
  {'name': 'Wall_SW_main',
   'direction': 'SW',
   'area': 10.1575,
   'space_type': 'main',
   'type': 'Wall',
   'structure': 'wood',
   'IsSunshadeInput': False,
   'UA': 6.67},
  {'name': 'Wall_SW_other',
   'direction': 'SW',
   'area': 10.1575,
   'space_type': 'other',
   'typ